In [1]:
!pwd

/home/giakhang/dev/pose_sandbox


In [2]:
import sys
import os
sys.path.append(os.path.join(os.path.abspath(os.curdir),
                "Hand_pose_estimation_3D/arm_and_hand"))
sys.path.append(os.path.join(os.path.abspath(os.curdir),
                "Hand_pose_estimation_3D"))

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
import torch.nn.functional as F
import matplotlib.pyplot as plt
import os
from datetime import datetime
import glob
from transformer_encoder import TransformerEncoder
from torch.optim.lr_scheduler import ReduceLROnPlateau
from ann import ANN
from dataloader_ann import HandArmLandmarksDataset_ANN
import math
from sklearn.preprocessing import MinMaxScaler
from csv_writer import columns_to_normalize, fusion_csv_columns_name
import pandas as pd
import joblib
import numpy as np

from landmarks_scaler import LandmarksScaler
from train_ann_no_intrinsics import train_model

In [3]:
INPUT_DIM = 144 + 144
OUTPUT_DIM = 144
HIDDEN_DIM = int(144 + (144 / 2))
NUM_HIDDEN_LAYERS = 4
DROPOUT_RATE = 0.1

model = ANN(input_dim=INPUT_DIM,
            output_dim=OUTPUT_DIM,
            hidden_dim=HIDDEN_DIM,
            num_hidden_layers=NUM_HIDDEN_LAYERS,
            dropout_rate=DROPOUT_RATE)
model = model.to("cuda")

In [4]:
MODEL_NAME = "ann"
DATETIME = "{}".format(datetime.now().strftime("%Y%m%d-%H%M"))
DATE = "{}".format(datetime.now().strftime("%Y%m%d"))
BASE_DIR = "/home/giakhang/dev/pose_sandbox/Hand_pose_estimation_3D/arm_and_hand/runs/{}".format(MODEL_NAME)
SAVE_DIR = os.path.join(BASE_DIR, DATE, DATETIME)
DATA_DIR = "/home/giakhang/dev/pose_sandbox/data"  
writer = SummaryWriter(log_dir=SAVE_DIR)

In [5]:
SELECTED_DATE = "2024-*"
train_paths = glob.glob(os.path.join(DATA_DIR, "{}/{}/fine_landmarks_{}_*.csv".format(SELECTED_DATE, SELECTED_DATE, "train")))
val_paths = glob.glob(os.path.join(DATA_DIR, "{}/{}/fine_landmarks_{}_*.csv".format(SELECTED_DATE, SELECTED_DATE, "val")))
body_lines = [[0,2], [0, 3], [2, 4], [3, 4]]
lefthand_lines = [[0, 1], [1, 5], [5, 6], 
                  [5, 10], [5, 22], [10, 14], 
                  [14, 18], [18, 22], [6, 7], 
                  [7, 8], [8, 9], [10, 11], 
                  [11, 12], [12, 13], [14, 15], 
                  [15, 16], [16, 17], [18, 19], 
                  [19, 20], [20, 21], [22, 23], 
                  [23, 24], [24, 25]]
train_body_distance_thres = 550
train_leftarm_distance_thres = 550
train_lefthand_distance_thres = 200
val_body_distance_thres=450,
val_leftarm_distance_thres=450,
val_lefthand_distance_thres=150,

In [6]:
# Load the true dataset to get the scaler then pass the scaler to the true and fake dataset
minmax_scaler = MinMaxScaler()
train_dataset = HandArmLandmarksDataset_ANN(train_paths, 
    body_lines, 
    lefthand_lines, 
    train_body_distance_thres, 
    train_leftarm_distance_thres, 
    train_lefthand_distance_thres,
    filter_outlier=True,
    only_keep_frames_contain_lefthand=True,
    cvt_normalized_xy_to_XY=True)
minmax_scaler.fit_transform(train_dataset._inputs)
scaler_save_path = os.path.join(SAVE_DIR, "input_scaler.pkl")
joblib.dump(minmax_scaler, scaler_save_path)

['/home/giakhang/dev/pose_sandbox/Hand_pose_estimation_3D/arm_and_hand/runs/ann/20241017/20241017-2032/input_scaler.pkl']

In [7]:
scaler = LandmarksScaler(columns_to_scale=columns_to_normalize,
    scaler_path=scaler_save_path)
train_dataset = HandArmLandmarksDataset_ANN(train_paths, 
    body_lines, 
    lefthand_lines, 
    train_body_distance_thres, 
    train_leftarm_distance_thres, 
    train_lefthand_distance_thres,
    filter_outlier=True,
    only_keep_frames_contain_lefthand=True,
    scaler=scaler,
    cvt_normalized_xy_to_XY=True)
train_dataloader = DataLoader(train_dataset, batch_size=256, shuffle=True)
val_dataset = HandArmLandmarksDataset_ANN(val_paths,
    body_lines,
    lefthand_lines,
    val_body_distance_thres,
    val_leftarm_distance_thres,
    val_lefthand_distance_thres,
    filter_outlier=True,
    only_keep_frames_contain_lefthand=True,
    scaler=scaler,
    cvt_normalized_xy_to_XY=True)
val_dataloader = DataLoader(val_dataset, batch_size=256, shuffle=True)  

In [8]:
optimizer = optim.Adam(model.parameters(), lr=1e-3)
num_epochs = 50000
current_time = datetime.now().strftime('%Y%m%d-%H%M')
save_path = os.path.join(SAVE_DIR, "{}_{}_layers_best.pth".format(MODEL_NAME, NUM_HIDDEN_LAYERS))
scheduler = ReduceLROnPlateau(optimizer, mode='min', 
    factor=math.sqrt(0.1), patience=1000, verbose=True, min_lr=1e-8)
early_stopping = None

/home/giakhang/py_venv/mmpose/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


In [9]:
left_wrist_idx = 5
num_left_arm_landmarks = 21
left_arm_idx = list(range(left_wrist_idx, left_wrist_idx + num_left_arm_landmarks))

train_losses, val_losses = train_model(model, 
    train_dataloader, 
    val_dataloader, 
    optimizer, 
    num_epochs=num_epochs, 
    save_path=save_path,
    early_stopping=early_stopping,
    scheduler=scheduler,
    writer=writer,
    log_seq=50,
    train_left_arm_hand_only=True,
    weight_idx=left_arm_idx,
    weight=2)

writer.close()

Model saved with Validation Loss: 2056706.3625
Model saved with Validation Loss: 2025156.8125
Model saved with Validation Loss: 2004030.7375
Model saved with Validation Loss: 1946722.8250
Model saved with Validation Loss: 1924188.3000
Model saved with Validation Loss: 1888404.0750
Model saved with Validation Loss: 1869705.0375
Model saved with Validation Loss: 1861003.5750
Model saved with Validation Loss: 1774888.8500
Model saved with Validation Loss: 1774573.0625
Epoch 50/50000, Training Loss: 2090552.1921
Epoch 50/50000, Validation Loss: 1747937.3500
Model saved with Validation Loss: 1747937.3500
Model saved with Validation Loss: 1718952.4125
Model saved with Validation Loss: 1669267.3500
Model saved with Validation Loss: 1647750.8000
Model saved with Validation Loss: 1534806.8625
Epoch 100/50000, Training Loss: 1955049.3933
Epoch 100/50000, Validation Loss: 1961238.5500
Model saved with Validation Loss: 1522058.6750
Model saved with Validation Loss: 1518691.2625
Epoch 150/50000, Tr

KeyboardInterrupt: 